### Displacement Invariant Transformer (**Simple** usage example)

for more complex usage, please check the documentation: *ToDo: add documentation*

In [1]:
import os
os.chdir("..")

from nifeatures.transform import DisplacementInvariantTransformer
from nifeatures.search import DisplacementInvariantTransformerCV
from nilearn.datasets import load_mni152_brain_mask
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [2]:
# Generate 100 simulated fMRI activation maps;
MNI = load_mni152_brain_mask(resolution=2)
template_data = MNI.get_fdata()
data = []
target = []

for sample in range(100):
    noisy_data = template_data.copy()
    weight = np.random.normal(0, 1, 1)[0]
    noisy_data[noisy_data == 1] = np.random.normal(0, 1, np.sum(template_data == 1)) * weight
    if sample == 0:
        data = noisy_data.flatten()
    else:
        data = np.vstack((data, noisy_data.flatten()))
    
    target.append(weight)


### Check if transformer works (for debugging purposes!)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, target)

dt = DisplacementInvariantTransformer(probability="softmax")
dt.fit(X_train, y_train)

/home/giuseppe/PNI/Projects/nifeatures/nifeatures/transform.py:103: UserWarning: No mask provided, using default mask (MNI152, 2mm).
  warnings.warn("No mask provided, using default mask (MNI152, 2mm).")


DisplacementInvariantTransformer(mask=<nibabel.nifti1.Nifti1Image object at 0x7fac9e82f950>,
                                 probability='softmax')

In [4]:
new_data = dt.transform(X_train)

# check shapes;
print("old shape: ", X_train.shape)
print("new shape: ", new_data.shape)

old shape:  (75, 1100385)
new shape:  (75, 31)


### Usage example without Hyperparameter Search

In [5]:
# The transformer is sklearn compatible.
# This means we can use it inside a sklearn pipeline:
model = Pipeline([
    ("trf", DisplacementInvariantTransformer()),
    ("model", Ridge())
])

# # Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = model.fit(X_train, y_train)

# Predict test values and score performance;
prediction = fit.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.0004747224554527177


### Usage example with Hyperparameter Search

The transformer class can be used in a scikit-learn pipeline, just like any other transformer. In this case, we can use search algorithms like GridSearchCV to perform hyperparameter tuning:

In [6]:
mask = load_mni152_brain_mask(resolution=2)

model = Pipeline([
    ("trf", DisplacementInvariantTransformer(mask=mask)),
    ("model", Ridge())
])

params = {
    "trf__n_peaks": [100, 200],
    "model__alpha": [0.1, 1.0]
}

# Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = GridSearchCV(model, params, cv=5, n_jobs=12).fit(X_train, y_train)

# Predict test values and score performance;
prediction = fit.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.0016468845429783528


There is also a dedicated class for hyperparameter search called "DisplacementInvariantTransformerCV". This class takes in a grid of parameters just like GridSearchCV, but it takes in just the estimator as instead of a scikit-learn pipeline. Data transformation is performed regardless.

In [5]:
model = Ridge # passed as a callable;
mask = load_mni152_brain_mask(resolution=2)

params = {
    "trf__n_peaks": [100, 200],
    "model__alpha": [0.1, 1.0]
}

# Fit the model;
X_train, X_test, y_train, y_test = train_test_split(data, target)
fit = DisplacementInvariantTransformerCV(
    model, 
    params, 
    mask=mask,
    shuffle = True,
    cv=5, 
    n_jobs=12).fit(X_train, y_train)

# Use the best model to predict test values and score performance;
# Note that the best model is not re-fitted by default;
prediction = fit.best_model_.predict(X_test)
print("R2: ", np.corrcoef(prediction, y_test)[0, 1]**2)

R2:  0.00059119406774539


DisplacementInvariantTransformerCV also returns a list of all combination of parameters used to fit the models with cross-validation and the list of the ROI-coordinates computed by the transformer before data transformation:

In [26]:
fit.coordinates_

,keys,values,score,coordinates,indices
0,"[trf__n_peaks, model__alpha]","(100, 0.1)",0.539483,"[[72, 47, 64], [42, 39, 38], [26, 73, 38], [39...","([0, 1, 3, 4, 5, 7, 8, 11, 12, 13, 14, 15, 16,..."
1,"[trf__n_peaks, model__alpha]","(100, 1.0)",0.607289,"[[72, 47, 64], [42, 39, 38], [26, 73, 38], [39...","([0, 1, 3, 4, 5, 7, 8, 11, 12, 13, 14, 15, 16,..."
2,"[trf__n_peaks, model__alpha]","(200, 0.1)",0.156859,"[[72, 47, 64], [42, 39, 38], [26, 73, 38], [39...","([0, 1, 3, 4, 5, 7, 8, 11, 12, 13, 14, 15, 16,..."
3,"[trf__n_peaks, model__alpha]","(200, 1.0)",0.310286,"[[72, 47, 64], [42, 39, 38], [26, 73, 38], [39...","([0, 1, 3, 4, 5, 7, 8, 11, 12, 13, 14, 15, 16,..."
4,"[trf__n_peaks, model__alpha]","(100, 0.1)",0.155810,"[[42, 39, 38], [66, 41, 28], [29, 37, 29], [26...","([1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16,..."
5,"[trf__n_peaks, model__alpha]","(100, 1.0)",0.136466,"[[42, 39, 38], [66, 41, 28], [29, 37, 29], [26...","([1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16,..."
6,"[trf__n_peaks, model__alpha]","(200, 0.1)",0.021837,"[[42, 39, 38], [66, 41, 28], [29, 37, 29], [26...","([1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16,..."
7,"[trf__n_peaks, model__alpha]","(200, 1.0)",0.086323,"[[42, 39, 38], [66, 41, 28], [29, 37, 29], [26...","([1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16,..."
8,"[trf__n_peaks, model__alpha]","(100, 0.1)",0.076242,"[[50, 22, 42], [32, 64, 42], [37, 61, 60], [49...","([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15..."
9,"[trf__n_peaks, model__alpha]","(100, 1.0)",0.056009,"[[50, 22, 42], [32, 64, 42], [37, 61, 60], [49...","([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15..."
